Compil ou Face
========

Notions de compilation pour le reverseur
--------------------------------------------

par *Serge Guelton*☘ et *Adrien Guinet*⚽

☘ ``sguelton@quarkslab.com``

⚽ ``aguinet@quarkslab.com``

``man sguelton``
=========

- Ingénieur R&D à Quarkslab, spécialisé en compilation (Python, LLVM)
- Chercheur associé à Télécom Bretagne

``man aguinet``
=========

- Ingénieur R&D à Quarkslab, spécialisé en tout

Ce Cours (il est long)
============

- Connaitre sa chaîne de compilation
- Connaître son compilo
- Comprendre quelques transformations et analyse

Cours et TP entrelacés

Intérêt pour le reverser
=============

- Mieux comprendre le code généré
- Billes de compréhension pour écrire des outils d'analyse
- Culture G

Chaîne de compilation classique
=================

```

a.c -- a.o --
             :-- a.out 
b.c -- b.o --
```

In [ ]:
%%file a.c
#include <stdio.h>
void greet(char const* who) {
    printf("Hello %s!\n", who);
}

In [ ]:
%%file b.c
extern void greet(char const* who);
int main(int argc, char const* argv[]) {
    if(argc == 1) greet("world");
    else greet(argv[1]);
    return 0;
}

In [ ]:
%%!
clang-3.7 a.c b.c
./a.out $USER

Pas à Pas : Préprocesseur
==============

*a.k.a.* « le sed du pauvre »

In [ ]:
!clang-3.7 -E a.c | wc -l

In [ ]:
!clang-3.7 -E a.c | head -n 10

Pas à Pas : La Représentation Interne
=====================

Généralement pas exposée à l'utilisateur…

In [ ]:
!clang-3.7 -S -emit-llvm a.c
!head -n 16 a.ll

Pas à Pas : La transformation de RI (olé !)
====================

Ce qui se cache derrière ``-O2`` et consorts

In [ ]:
!opt-3.7 -mem2reg a.ll -S | head -n 10

Pas à Pas : La génération de code assembleur
========================

Avec la bonne variante syntaxique…

In [ ]:
!llc-3.7 a.ll --x86-asm-syntax=intel
!head -n 23 a.s | tail -n 10

Ou plus simplement :



In [ ]:
!clang-3.7 -S  a.c #-masm=intel if needed

Pas à Pas : L'assemblage
==============

Génération de code objet, ou ``.o``

In [ ]:
!as a.s -o a.o
!file a.o

Pas à Pas : L'édition de liens
=================

Le *linker*, les libs statiques, les lib dynamiques…

In [ ]:
!clang-3.7 -c b.c # pour avoir le deuxième code objet

In [ ]:
!nm b.o | grep greet

In [ ]:
!nm a.o | grep greet
!nm a.o | grep printf

In [ ]:
!ldd /bin/ls | grep libc.so

In [ ]:
!nm /lib/x86_64-linux-gnu/libc.so.6 | grep printf

In [ ]:
!file -L /lib/x86_64-linux-gnu/libc.so.6

In [ ]:
!readelf -s /lib/x86_64-linux-gnu/libc.so.6 | grep ' printf'

Pas à Pas : l'exécutable
==============

Pour pondre un joli petit ``a.out`` tout mignon

In [ ]:
! ld a.o b.o && ./a.out

In [ ]:
! ld a.o b.o -lc && file a.out && ./a.out

In [ ]:
! clang-3.7 -v a.o b.o 2>&1 | grep ld

In [ ]:
! ./a.out

Comprendre l'*Abstract Syntax Tree*
===================

Jouons avec Python et son AST, plus facile que celui de C++

In [ ]:
import ast
tree = ast.parse("print(1)")
print(tree)

In [ ]:
ast.dump(tree)

In [ ]:
import astdump
astdump.indented(tree)

Compilation de l'AST
===========

Passage d'une représentation proche du langage à une représentation proche de l'interpréteur.

In [ ]:
code = compile(tree, '<>', 'exec')

In [ ]:
eval(code)

Inspection du bytecode
=============

CPython → Interpréteur à pile

In [ ]:
import dis
dis.dis(code)

``(`` Aparté
========

D'après vous, quels sont les avantages et inconvénients d'une interpréteur à pile par rapport à un interpréteur à registre?

Interpréteur à pile
----------------------

Facile de conception, peu d'optimisations

Interpréteur à registre
--------------------------

Plus complexe (et pas seulement pour l'allocation de registre) mais permet de modéliser plus d'optimisations

Game of Stack
=======

Écrire un interpréteur (dans le langage de votre choix) qui comprend les instructions suivantes :

- ``PUSH <integer>`` qui ajoute ``<integer>`` au dessus de la pile
- ``DUP`` qui duplique le dessus de la pile
- ``ADD`` qui enlève les deux premiers éléments de la pile et ajoute ``S[0] + S[1]`` au dessus de la pile
- ``MUL`` qui enlève les deux premiers éléments de la pile et ajoute ``S[0] * S[1]`` au dessus de la pile
- ``READ`` qui lit un entier sur ``stdin`` et l'ajoute au dessus de la pile
- ``WRITE`` qui dépile le premier élément de la pile et l'affiche sudr ``stdout``


Par exemple :

```
0 READ
1 DUP
2 ADD
3 WRITE
```


Introduisons maintenant une optmisation (de ouf !). Les deux séquences suivantes sont équivalentes :

```
PUSH 2
MUL
```

et

```
DUP
ADD
```

Ajoutez à votre interpréteur une passe qui effectue de qui s'avère être une *peephole optimisation* en transformant l'une en l'autre.


Fin de l'aparté ``)``
==========

Continuons à jouer avec l'AST
================

L'AST Python peut être parcouru grâce à un **visiteur** (qui n'est pas né d'hier)

À lire : https://docs.python.org/3/library/ast.html

In [ ]:
class VisitIntegers(ast.NodeVisitor):
    def visit_Num(self, node):
        if isinstance(node.n, int):
            print(node.n)

VisitIntegers().visit(tree)

Exo
===

Écrivez un visiteur qui va trouver tous les appels à la fonction ``open``

Pourquoi est-ce en fait impossible en analyse statique ?

En instrumentant
=========

Une sorte d'analyse dynamique ?

In [ ]:
import builtins
real_open = builtins.open
def myopen(*args, **kwargs):
    print(args, kwargs)
    return real_open(*args, **kwargs)
builtins.open = myopen
open("/dev/null")

In [ ]:
builtins.open = real_open

Comprendre la Représentation Interne (IR dans la langue de Homer)
=======================

En se basant sur celle de LLVM.

Étudions plus précisément la sortie de clang sur un code simple :

In [ ]:
%%file mask.c
void mask(unsigned n, int data[n], int mask) {
    for(unsigned i = 0; i < n; ++i)
        data[i] ^= mask;
}

In [ ]:
! clang-3.7 -O1 -S -emit-llvm -o - mask.c | sed -n '6,18 p'

Représentation Hiérarchique
===============

Un Module contient :
- des variables globales
- des fonctions
- des métadonnées

Représentation Hiérarchique
===============

Une fonction contient des blocs de base…

Qui contiennent des instructions…

Qui finissent par un…

Terminator
===============

- Ils terminent (d'oh) un bloc de base
- Transfèrent le flot de contrôle à un autre bloc (``br``) à l'appelant (``ret``), à un gestionnaire d'exception (``invoke``) ou à rien (``unreachable``

```
                   <((((((\\\
                   /      . }\
                   ;--..--._|}
(\                 '--/\--'  )
 \\                | '-'  :'|
  \\               . -==- .-|
   \\               \.__.'   \--._
   [\\          __.--|       //  _/'--.
   \ \\       .'-._ ('-----'/ __/      \
    \ \\     /   __>|      | '--.       |
     \ \\   |   \   |     /    /       /
      \ '\ /     \  |     |  _/       /
       \  \       \ |     | /        /
 snd    \  \      \        /
 ```
 
 source: http://www.ascii-code.com/ascii-art/movies/other.php

Les instructions
=========

- Produisent et utilisent des **valeurs** stockées dans des **registres virtuels**

```
%4 = xor i32 %3, %mask
```

- Manipulent des valeurs fortement typées :

```
%2 = getelementptr inbounds i32, i32* %data, i64 %indvars.iv
```

- Calcul, accès à la pile, appel de fonction, conversions…

- Peuvent être annotées (*metadata*)

```
%3 = load i32, i32* %2, align 4, !tbaa !1
...
!1 = !{!2, !2, i64 0}
!2 = !{!"int", !3, i64 0}
```

Forme *Single Static Assignment*
==================

- un registre virtuel ``%stuff`` n'est écrit qu'une fois
- la mémoire peut être lue ou écrite à travers ``load`` et ``store``
- l'instruction ``phi`` permet de faire un choix suivant le prédécesseur

```
%indvars.iv = phi i64 [ %indvars.iv.next, %.lr.ph ], [ 0, %0 ]
```

Avantages de cette réprésentation
================

- Propagation de constante qui rend cette situation impossible

```
%0 = i32 8
%1 = add i32 %param, %0
%2 = mul i32 %1, %0
```

- Élimination de code mort (bloc de base sans prédecesseur)

Et pleins de choses qui vont au delà de ce cours…

JIT do it
======

Les compilateurs à la volée permettent de :

- construire en mémoire une représentation du programme
- modifier cette représentation en fonction du contexte
- traduire cette représentation en code machine
- charger (incl. édition de lien) ce code dans l'espace mémoire du processus courant 


Exemple de JIT
==============

Le JIT de LLVM

In [ ]:
!sed -n '/extern/,/BasicBloc/ p' interp.cpp

Exercice
=====

Étendez le code (téléchargeable sur https://gist.github.com/serge-sans-paille/aa332fa22692fcdfdc51) pour supporter plus d'opérations !

Comment le compilateur voit…
==================

Différents concepts de haut niveau passés à la moulinette

La vectorisation
==========

TL;DR : un registre vectoriel contient plusieurs scalaires et les opérations sur un registre vectoriel affectent chaque scalaire

```
[a0     [b0     [c0
 a1  +   b1  =   c1
 a2      b2      c2
 a3]     b3]     c3]
```

In [ ]:
!clang-3.7 -O3 -march=native mask.c -S -emit-llvm -o - | sed -n '/vector.body:/,/xor <8/ p'

Une initialisation de structure
=================

In [ ]:
%%file init_struct.c
struct foo { int a; float b; char c ;};
struct foo toir() {
    struct foo res = {.a = 0, .b =2.5, .c=3};
    return res;
}

In [ ]:
!clang-3.7 -S -emit-llvm -O2 -o - init_struct.c | head -n 8

Une classe avec méthode
=============

In [ ]:
%%file class_with_method.cpp
class ik {
  public:
    int field;
    ik(int f) : field(f) {}
};
ik ariwarrior(ik const& ipe) {
    
    return ik(ipe.field);
}

In [ ]:
!clang++-3.7 -O1 -S -std=c++11 -emit-llvm class_with_method.cpp -o - | grep "%class.*type"

In [ ]:
!clang++-3.7 -O1 -S -std=c++11 -emit-llvm class_with_method.cpp -o - | grep "comdat"

In [ ]:
!clang++-3.7 -O1 -S -std=c++11 -emit-llvm class_with_method.cpp -o - | grep this

In [ ]:
!printf _ZN2ikC2Ei | c++filt

Un ``memcpy``
========

In [ ]:
%%file memcpy.c
#include <string.h>
void fast(size_t n, long *bacon, long const *antinople) {
    memcpy(bacon, antinople, sizeof(long) * n);
}

In [ ]:
!clang-3.7 -S -emit-llvm memcpy.c -O2 -o - | sed -n '5,12 p'

Note
====

Dans ``@llvm.memcpy.p0i8.p0i8.i64(i8* %1, i8* %2, i64 %3, i32 8, i1 false)``, le ``false`` indique un accès non volatile, koike ça vous évoque ?

Une levée d'exception
============


In [ ]:
%%file raise.cpp
extern void foo();
void bar() {
    try { foo(); }
    catch(float) {}
}

In [ ]:
!clang++-3.7 -S -emit-llvm -O2 -o - raise.cpp | sed -n '/invoke/,/resume/ p'